# Predicte 2025 sales: test on different models

In [10]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_absolute_error,
    root_mean_squared_error,
    r2_score,
    median_absolute_error,
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# =========================================================
# 0. 全局配置
# =========================================================
FILE_PATH = "../data/processed/bmw_sales_2020_2024_clean.csv"

CAT_FEATURES = ["model", "region", "fuel_type", "transmission", "color"]
NUM_FEATURES = ["year", "engine_size_l", "mileage_km", "price_usd"]
FEATURE_COLS = CAT_FEATURES + NUM_FEATURES

plt.rcParams["figure.dpi"] = 120


# =========================================================
# 1. 聚合原始数据（按年 + 类别组合）
# =========================================================
def load_and_aggregate(path: str) -> pd.DataFrame:
    """
    读取原始数据，并按
    year + model + region + fuel_type + transmission + color 聚合。

    - engine_size_l、mileage_km、price_usd 取均值
    - sales_volume 取和
    """
    df = pd.read_csv(path)

    group_cols = ["year", "model", "region", "fuel_type", "transmission", "color"]

    agg_df = (
        df.groupby(group_cols, as_index=False)
        .agg(
            {
                "engine_size_l": "mean",
                "mileage_km": "mean",
                "price_usd": "mean",
                "sales_volume": "sum",
            }
        )
    )

    return agg_df


# =========================================================
# 2. 通用指标函数
# =========================================================
def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    """
    通用指标计算：MAE / RMSE / MedianAE / MAPE / R2
    以及相对于真实值均值的百分率 MAE_pct / RMSE_pct / MedianAE_pct
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    med_ae = median_absolute_error(y_true, y_pred)

    # 避免除 0
    mask = y_true != 0
    if mask.sum() > 0:
        mape = (np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])).mean() * 100
    else:
        mape = np.nan

    if len(y_true) > 1:
        r2 = r2_score(y_true, y_pred)
    else:
        r2 = np.nan  # 单点 R2 没有意义

    mean_y = y_true.mean()
    mae_pct = mae / mean_y * 100 if mean_y != 0 else np.nan
    rmse_pct = rmse / mean_y * 100 if mean_y != 0 else np.nan
    med_ae_pct = med_ae / mean_y * 100 if mean_y != 0 else np.nan

    return {
        "MAE": mae,
        "RMSE": rmse,
        "MedianAE": med_ae,
        "MAPE": mape,
        "R2": r2,
        "MAE_pct": mae_pct,
        "RMSE_pct": rmse_pct,
        "MedianAE_pct": med_ae_pct,
    }


# =========================================================
# 3. Lag 特征构造（面向机器学习模型）
# =========================================================
def add_lag_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    为每个组合（model+region+fuel_type+transmission+color）增加 lag 特征：
    - sales_last1, sales_last2
    - sales_avg3
    - sales_growth
    - price_last1, price_trend
    - mileage_last1, mileage_trend
    """
    df = df.sort_values(
        ["model", "region", "fuel_type", "transmission", "color", "year"]
    ).copy()

    group_cols = ["model", "region", "fuel_type", "transmission", "color"]

    df["sales_last1"] = df.groupby(group_cols)["sales_volume"].shift(1)
    df["sales_last2"] = df.groupby(group_cols)["sales_volume"].shift(2)

    df["sales_avg3"] = df.groupby(group_cols)["sales_volume"].transform(
        lambda x: x.rolling(3).mean()
    )

    df["sales_growth"] = (df["sales_last1"] - df["sales_last2"]) / df["sales_last2"]

    df["price_last1"] = df.groupby(group_cols)["price_usd"].shift(1)
    df["price_trend"] = (df["price_usd"] - df["price_last1"]) / df["price_last1"]

    df["mileage_last1"] = df.groupby(group_cols)["mileage_km"].shift(1)
    df["mileage_trend"] = (df["mileage_km"] - df["mileage_last1"]) / df["mileage_last1"]

    # 更新数值特征列表
    lag_num_features = [
        "sales_last1",
        "sales_last2",
        "sales_avg3",
        "sales_growth",
        "price_last1",
        "price_trend",
        "mileage_last1",
        "mileage_trend",
    ]

    global NUM_FEATURES, FEATURE_COLS
    NUM_FEATURES = ["year", "engine_size_l", "mileage_km", "price_usd"] + lag_num_features
    FEATURE_COLS = CAT_FEATURES + NUM_FEATURES

    return df


# =========================================================
# 4. 构建各模型
# =========================================================
def build_xgb_pipeline():
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_FEATURES),
            ("num", "passthrough", NUM_FEATURES),
        ]
    )

    xgb = XGBRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
    )

    return Pipeline(steps=[("preprocess", preprocessor), ("model", xgb)])


def build_lgbm_pipeline():
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_FEATURES),
            ("num", "passthrough", NUM_FEATURES),
        ]
    )

    lgbm = LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=-1,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        objective="regression",
        verbose=-1,
    )

    return Pipeline(steps=[("preprocess", preprocessor), ("model", lgbm)])


def build_rf_pipeline():
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_FEATURES),
            ("num", "passthrough", NUM_FEATURES),
        ]
    )

    rf = RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1,
    )

    return Pipeline(steps=[("preprocess", preprocessor), ("model", rf)])


def build_catboost_model():
    """
    CatBoost 直接使用 DataFrame + cat_features 索引
    """
    return CatBoostRegressor(
        depth=8,
        learning_rate=0.05,
        n_estimators=800,
        loss_function="RMSE",
        eval_metric="RMSE",
        random_seed=42,
        verbose=False,
    )


# =========================================================
# 5. 训练 + 验证（2020-2023 训练，2024 验证）
# =========================================================
def train_and_validate_models(df_lagged: pd.DataFrame):
    """
    用带 lag 特征的数据，训练 XGB / LGBM / RF / CatBoost，并在 2024 做验证。
    返回：
      - metrics_all：字典，包含各模型的指标
      - 已训练好的 CatBoost（后续可选直接用）
    """
    # 删除 lag 特征或标签为 NaN 的行
    df_clean = df_lagged.dropna(subset=FEATURE_COLS + ["sales_volume"]).copy()

    # 按年份切分
    train_df = df_clean[df_clean["year"] < 2024]
    valid_df = df_clean[df_clean["year"] == 2024]

    X_train = train_df[FEATURE_COLS]
    y_train = train_df["sales_volume"].values

    X_valid = valid_df[FEATURE_COLS]
    y_valid = valid_df["sales_volume"].values

    print(
        f"Shape train (with lag): {X_train.shape}, "
        f"Shape valid (with lag): {X_valid.shape}\n"
    )

    metrics_all = {}

    # ---------- XGBoost ----------
    xgb_model = build_xgb_pipeline()
    xgb_model.fit(X_train, y_train)
    y_pred_xgb = xgb_model.predict(X_valid)
    metrics_xgb = compute_metrics(y_valid, y_pred_xgb)
    metrics_all["xgb"] = metrics_xgb

    print("Model: XGBoost")
    print(
        f"  MAE      : {metrics_xgb['MAE']:.2f} ({metrics_xgb['MAE_pct']:.2f}%)\n"
        f"  RMSE     : {metrics_xgb['RMSE']:.2f} ({metrics_xgb['RMSE_pct']:.2f}%)\n"
        f"  MedianAE : {metrics_xgb['MedianAE']:.2f} ({metrics_xgb['MedianAE_pct']:.2f}%)\n"
        f"  MAPE     : {metrics_xgb['MAPE']:.2f}%\n"
        f"  R2       : {metrics_xgb['R2']:.4f}\n"
    )

    # ---------- LightGBM ----------
    lgbm_model = build_lgbm_pipeline()
    lgbm_model.fit(X_train, y_train)
    y_pred_lgbm = lgbm_model.predict(X_valid)
    metrics_lgbm = compute_metrics(y_valid, y_pred_lgbm)
    metrics_all["lgbm"] = metrics_lgbm

    print("Model: LightGBM")
    print(
        f"  MAE      : {metrics_lgbm['MAE']:.2f} ({metrics_lgbm['MAE_pct']:.2f}%)\n"
        f"  RMSE     : {metrics_lgbm['RMSE']:.2f} ({metrics_lgbm['RMSE_pct']:.2f}%)\n"
        f"  MedianAE : {metrics_lgbm['MedianAE']:.2f} ({metrics_lgbm['MedianAE_pct']:.2f}%)\n"
        f"  MAPE     : {metrics_lgbm['MAPE']:.2f}%\n"
        f"  R2       : {metrics_lgbm['R2']:.4f}\n"
    )

    # ---------- RandomForest ----------
    rf_model = build_rf_pipeline()
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_valid)
    metrics_rf = compute_metrics(y_valid, y_pred_rf)
    metrics_all["rf"] = metrics_rf

    print("Model: RandomForest")
    print(
        f"  MAE      : {metrics_rf['MAE']:.2f} ({metrics_rf['MAE_pct']:.2f}%)\n"
        f"  RMSE     : {metrics_rf['RMSE']:.2f} ({metrics_rf['RMSE_pct']:.2f}%)\n"
        f"  MedianAE : {metrics_rf['MedianAE']:.2f} ({metrics_rf['MedianAE_pct']:.2f}%)\n"
        f"  MAPE     : {metrics_rf['MAPE']:.2f}%\n"
        f"  R2       : {metrics_rf['R2']:.4f}\n"
    )

    # ---------- CatBoost ----------
    cat_model = build_catboost_model()
    cat_feature_indices = [X_train.columns.get_loc(col) for col in CAT_FEATURES]

    cat_model.fit(X_train, y_train, cat_features=cat_feature_indices)
    y_pred_cat = cat_model.predict(X_valid)
    metrics_cat = compute_metrics(y_valid, y_pred_cat)
    metrics_all["catboost"] = metrics_cat

    print("Model: CatBoost")
    print(
        f"  MAE      : {metrics_cat['MAE']:.2f} ({metrics_cat['MAE_pct']:.2f}%)\n"
        f"  RMSE     : {metrics_cat['RMSE']:.2f} ({metrics_cat['RMSE_pct']:.2f}%)\n"
        f"  MedianAE : {metrics_cat['MedianAE']:.2f} ({metrics_cat['MedianAE_pct']:.2f}%)\n"
        f"  MAPE     : {metrics_cat['MAPE']:.2f}%\n"
        f"  R2       : {metrics_cat['R2']:.4f}\n"
    )

    return metrics_all, cat_model


# =========================================================
# 6. 2025 特征构造 + CatBoost 预测
# =========================================================
def build_2025_features_from_lagged(df_lagged: pd.DataFrame) -> pd.DataFrame:
    """
    利用到 2024 年的 lag 特征，构造 2025 年的特征行：
    - sales_last1_2025 = 2024 的 sales_volume
    - sales_last2_2025 = 2024 的 sales_last1
    - sales_avg3_2025  = (2024, 2023, 2022) 三年平均
    价格、里程的 last1 / trend 沿用 2024。
    """
    group_cols = ["model", "region", "fuel_type", "transmission", "color"]

    df_sorted = df_lagged.sort_values(group_cols + ["year"])
    last_rows = df_sorted.groupby(group_cols, as_index=False).tail(1)

    # 过滤必须有完整 lag 信息的组合
    last_rows = last_rows.dropna(
        subset=["sales_last1", "sales_last2", "sales_avg3", "price_last1", "mileage_last1"]
    )

    rows_2025 = []

    for _, row in last_rows.iterrows():
        new_row = row.copy()
        new_row["year"] = 2025

        sales_last1_2025 = row["sales_volume"]
        sales_last2_2025 = row["sales_last1"]
        sales_avg3_2025 = (
            row["sales_volume"] + row["sales_last1"] + row["sales_last2"]
        ) / 3

        new_row["sales_last1"] = sales_last1_2025
        new_row["sales_last2"] = sales_last2_2025
        new_row["sales_avg3"] = sales_avg3_2025

        if not pd.isna(sales_last2_2025) and sales_last2_2025 != 0:
            new_row["sales_growth"] = (
                sales_last1_2025 - sales_last2_2025
            ) / sales_last2_2025
        else:
            new_row["sales_growth"] = 0.0

        new_row["price_last1"] = row["price_usd"]
        new_row["mileage_last1"] = row["mileage_km"]

        # 沿用原来的趋势
        new_row["price_trend"] = row.get("price_trend", 0.0)
        new_row["mileage_trend"] = row.get("mileage_trend", 0.0)

        # 2025 年真实销量未知
        new_row["sales_volume"] = np.nan

        rows_2025.append(new_row)

    feat_2025 = pd.DataFrame(rows_2025)
    # 确保列顺序与 FEATURE_COLS 一致
    feat_2025 = feat_2025[[c for c in feat_2025.columns if c not in ["sales_volume"]] + ["sales_volume"]]

    return feat_2025


def retrain_catboost_and_predict_2025(df_lagged: pd.DataFrame):
    """
    使用 2020-2024 全部数据重训 CatBoost，
    对构造出的 2025 特征进行预测，并按多个维度聚合。
    """
    df_clean = df_lagged.dropna(subset=FEATURE_COLS + ["sales_volume"]).copy()

    X_full = df_clean[FEATURE_COLS]
    y_full = df_clean["sales_volume"].values

    feat_2025 = build_2025_features_from_lagged(df_lagged)
    X_2025 = feat_2025[FEATURE_COLS]

    cat_model = build_catboost_model()
    cat_feature_indices = [X_full.columns.get_loc(col) for col in CAT_FEATURES]
    cat_model.fit(X_full, y_full, cat_features=cat_feature_indices)

    preds_2025 = cat_model.predict(X_2025)
    total_2025 = preds_2025.sum()

    feat_2025_out = feat_2025.copy()
    feat_2025_out["pred_sales_volume_2025_catboost"] = preds_2025

    print(f"\n=== CatBoost predicted total sales volume in 2025: {total_2025:.2f} ===\n")

    # 按不同维度聚合
    col_pred = "pred_sales_volume_2025_catboost"

    by_model = (
        feat_2025_out.groupby("model")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )
    by_region = (
        feat_2025_out.groupby("region")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )
    by_fuel = (
        feat_2025_out.groupby("fuel_type")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )
    by_trans = (
        feat_2025_out.groupby("transmission")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )
    by_color = (
        feat_2025_out.groupby("color")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )

    return total_2025, feat_2025_out, by_model, by_region, by_fuel, by_trans, by_color


# =========================================================
# 7. 指标对比表 + 单张柱状图
# =========================================================
def build_metrics_table(metrics_all: dict) -> pd.DataFrame:
    """
    构造成 DataFrame，对比 XGB / LGBM / RF / CatBoost 的指标。
    """
    df = pd.DataFrame.from_dict(metrics_all, orient="index")
    df.index.name = "model"
    df = df.loc[["xgb", "lgbm", "rf", "catboost"]]
    return df


def plot_metrics_bar(metrics_df: pd.DataFrame, save_path: str = None):
    """
    画一张柱状图，对比不同模型在 4 个指标上的表现：
    - MAE_pct
    - RMSE_pct
    - MedianAE_pct
    - MAPE（本来就是百分比）
    """
    models = metrics_df.index.tolist()
    x = np.arange(len(models))

    metrics_to_plot = ["MAE_pct", "RMSE_pct", "MedianAE_pct", "MAPE"]
    colors = ["#1C69D4", "#4C8BF5", "#82B1FF", "#FFB300"]
    width = 0.18

    fig, ax = plt.subplots(figsize=(10, 5))

    for i, (metric, c) in enumerate(zip(metrics_to_plot, colors)):
        vals = metrics_df[metric].values
        ax.bar(x + i * width - 1.5 * width, vals, width, label=metric, color=c)

    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=15)
    ax.set_ylabel("Error / Percentage")
    ax.set_title("Model Performance Comparison on 2024 (with lag features)")

    ax.legend()
    ax.grid(axis="y", linestyle="--", alpha=0.3)

    fig.tight_layout()
    if save_path:
        fig.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close(fig)


# =========================================================
# 8. 主流程
# =========================================================
if __name__ == "__main__":
    # 1. 读取 & 聚合
    agg_df = load_and_aggregate(FILE_PATH)
    print("Aggregated shape (before lag):", agg_df.shape)
    print(agg_df.head(), "\n")

    # 2. 加 lag 特征
    agg_df_lagged = add_lag_features(agg_df)
    print(
        "Shape after adding lag features (before dropna):",
        agg_df_lagged.shape,
        "\n",
    )

    # 3. 训练 + 验证（2020-2023 训练，2024 验证）
    metrics_all, cat_model_trained = train_and_validate_models(agg_df_lagged)

    # 4. 对比表格（含百分率）
    metrics_df = build_metrics_table(metrics_all)
    print("\n=== Model comparison on 2024 validation (with lag, row-level) ===")
    print(metrics_df, "\n")

    # 5. 一张柱状图比较不同指标
    plot_metrics_bar(metrics_df, save_path="model_performance_2024_with_lag.png")
    print("Saved bar chart to: model_performance_2024_with_lag.png\n")

    # 6. 用 CatBoost 预测 2025 总销量 & 分组销量
    (
        total_2025_cat,
        df_cat_2025,
        by_model_2025,
        by_region_2025,
        by_fuel_2025,
        by_trans_2025,
        by_color_2025,
    ) = retrain_catboost_and_predict_2025(agg_df_lagged)

    print("CatBoost - 2025 predicted sales by model:")
    print(by_model_2025.head(), "\n")

    print("CatBoost - 2025 predicted sales by region:")
    print(by_region_2025, "\n")

    print("CatBoost - 2025 predicted sales by fuel_type:")
    print(by_fuel_2025, "\n")

    print("CatBoost - 2025 predicted sales by transmission:")
    print(by_trans_2025, "\n")

    print("CatBoost - 2025 predicted sales by color:")
    print(by_color_2025, "\n")

    print(f"CatBoost (row-level) predicted 2025 total sales: {total_2025_cat:.2f}")


Aggregated shape (before lag): (10306, 10)
   year     model  region fuel_type transmission   color  engine_size_l  \
0  2020  3 Series  Africa    Diesel    Automatic   Black           2.90   
1  2020  3 Series  Africa    Diesel    Automatic    Blue           3.00   
2  2020  3 Series  Africa    Diesel    Automatic     Red           3.55   
3  2020  3 Series  Africa    Diesel    Automatic  Silver           3.30   
4  2020  3 Series  Africa    Diesel       Manual   Black           4.60   

   mileage_km  price_usd  sales_volume  
0     54293.0   100920.0          8279  
1     22846.0    79435.5          6282  
2     85078.5   106306.0         14847  
3    182077.0    59951.0          2355  
4    107799.0    75078.0          3975   

Shape after adding lag features (before dropna): (10306, 18) 

Shape train (with lag): (2343, 17), Shape valid (with lag): (1808, 17)

Model: XGBoost
  MAE      : 789.90 (9.41%)
  RMSE     : 1181.66 (14.08%)
  MedianAE : 575.95 (6.86%)
  MAPE     : 29.28%
  

KeyboardInterrupt: 

# Comparing to Arima

In [ ]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_absolute_error,
    root_mean_squared_error,
    r2_score,
    median_absolute_error,
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from statsmodels.tsa.arima.model import ARIMA

import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# =========================================================
# 0. 全局配置
# =========================================================
FILE_PATH = "../data/processed/bmw_sales_2020_2024_clean.csv"

CAT_FEATURES = ["model", "region", "fuel_type", "transmission", "color"]
NUM_FEATURES = ["year", "engine_size_l", "mileage_km", "price_usd"]
FEATURE_COLS = CAT_FEATURES + NUM_FEATURES

plt.rcParams["figure.dpi"] = 120


# =========================================================
# 1. 聚合原始数据（按年 + 类别组合）
# =========================================================
def load_and_aggregate(path: str) -> pd.DataFrame:
    """
    读取原始数据，并按
    year + model + region + fuel_type + transmission + color 聚合。

    - engine_size_l、mileage_km、price_usd 取均值
    - sales_volume 取和
    """
    df = pd.read_csv(path)

    group_cols = ["year", "model", "region", "fuel_type", "transmission", "color"]

    agg_df = (
        df.groupby(group_cols, as_index=False)
        .agg(
            {
                "engine_size_l": "mean",
                "mileage_km": "mean",
                "price_usd": "mean",
                "sales_volume": "sum",
            }
        )
    )

    return agg_df


# =========================================================
# 2. ARIMA 年度总销量基线
# =========================================================
def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    """
    通用指标计算：MAE / RMSE / MedianAE / MAPE / R2
    以及相对于真实值均值的百分率 MAE_pct / RMSE_pct / MedianAE_pct
    """
    mae = mean_absolute_error(y_true, y_pred)
    # rmse = root_mean_squared_error(y_true, y_pred, squared=False)
    rmse = root_mean_squared_error(y_true, y_pred)
    med_ae = median_absolute_error(y_true, y_pred)

    # 避免除 0
    mask = y_true != 0
    if mask.sum() > 0:
        mape = (np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])).mean() * 100
    else:
        mape = np.nan

    if len(y_true) > 1:
        r2 = r2_score(y_true, y_pred)
    else:
        r2 = np.nan  # 单点 R2 没有意义

    mean_y = y_true.mean()
    mae_pct = mae / mean_y * 100 if mean_y != 0 else np.nan
    rmse_pct = rmse / mean_y * 100 if mean_y != 0 else np.nan
    med_ae_pct = med_ae / mean_y * 100 if mean_y != 0 else np.nan

    return {
        "MAE": mae,
        "RMSE": rmse,
        "MedianAE": med_ae,
        "MAPE": mape,
        "R2": r2,
        "MAE_pct": mae_pct,
        "RMSE_pct": rmse_pct,
        "MedianAE_pct": med_ae_pct,
    }


def arima_baseline_yearly(df_agg: pd.DataFrame):
    """
    用年度总销量做 ARIMA 基线：
    - 训练：2020-2023
    - 测试：2024
    - 再预测：2025 总销量
    """
    df_yearly = (
        df_agg.groupby("year")["sales_volume"].sum().sort_index()
    )  # Series: index=year, value=total sales

    print("=== ARIMA baseline on yearly total sales ===")
    print(df_yearly, "\n")

    train_series = df_yearly[df_yearly.index < 2024]
    test_value = df_yearly.loc[2024]

    # 简单 ARIMA(1,1,1)
    model = ARIMA(train_series, order=(1, 1, 1))
    fitted = model.fit()

    # 预测 2024（作为测试）
    pred_2024 = float(fitted.forecast(steps=1).iloc[0])
    print(f"ARIMA predicted 2024 total sales: {pred_2024:.2f}")
    print(f"Actual 2024 total sales        : {float(test_value):.2f}")

    metrics_2024 = compute_metrics(
        np.array([test_value], dtype=float), np.array([pred_2024], dtype=float)
    )

    print("ARIMA metrics (yearly total, 2024 as test):")
    for k, v in metrics_2024.items():
        print(f"  {k}: {v:.4f}")
    print()

    # 再往前预测一步得到 2025
    pred_2025 = float(fitted.forecast(steps=2).iloc[-1])
    print(f"=== ARIMA predicted total sales in 2025 (yearly total): {pred_2025:.2f} ===\n")

    return metrics_2024, pred_2025


# =========================================================
# 3. Lag 特征构造（面向机器学习模型）
# =========================================================
def add_lag_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    为每个组合（model+region+fuel_type+transmission+color）增加 lag 特征：
    - sales_last1, sales_last2
    - sales_avg3
    - sales_growth
    - price_last1, price_trend
    - mileage_last1, mileage_trend
    """
    df = df.sort_values(
        ["model", "region", "fuel_type", "transmission", "color", "year"]
    ).copy()

    group_cols = ["model", "region", "fuel_type", "transmission", "color"]

    df["sales_last1"] = df.groupby(group_cols)["sales_volume"].shift(1)
    df["sales_last2"] = df.groupby(group_cols)["sales_volume"].shift(2)

    df["sales_avg3"] = df.groupby(group_cols)["sales_volume"].transform(
        lambda x: x.rolling(3).mean()
    )

    df["sales_growth"] = (df["sales_last1"] - df["sales_last2"]) / df["sales_last2"]

    df["price_last1"] = df.groupby(group_cols)["price_usd"].shift(1)
    df["price_trend"] = (df["price_usd"] - df["price_last1"]) / df["price_last1"]

    df["mileage_last1"] = df.groupby(group_cols)["mileage_km"].shift(1)
    df["mileage_trend"] = (df["mileage_km"] - df["mileage_last1"]) / df["mileage_last1"]

    # 更新数值特征列表
    lag_num_features = [
        "sales_last1",
        "sales_last2",
        "sales_avg3",
        "sales_growth",
        "price_last1",
        "price_trend",
        "mileage_last1",
        "mileage_trend",
    ]

    global NUM_FEATURES, FEATURE_COLS
    NUM_FEATURES = ["year", "engine_size_l", "mileage_km", "price_usd"] + lag_num_features
    FEATURE_COLS = CAT_FEATURES + NUM_FEATURES

    return df


# =========================================================
# 4. 构建各模型
# =========================================================
def build_xgb_pipeline():
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_FEATURES),
            ("num", "passthrough", NUM_FEATURES),
        ]
    )

    xgb = XGBRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
    )

    return Pipeline(steps=[("preprocess", preprocessor), ("model", xgb)])


def build_lgbm_pipeline():
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_FEATURES),
            ("num", "passthrough", NUM_FEATURES),
        ]
    )

    lgbm = LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=-1,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        objective="regression",
        verbose=-1,
    )

    return Pipeline(steps=[("preprocess", preprocessor), ("model", lgbm)])


def build_rf_pipeline():
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_FEATURES),
            ("num", "passthrough", NUM_FEATURES),
        ]
    )

    rf = RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1,
    )

    return Pipeline(steps=[("preprocess", preprocessor), ("model", rf)])


def build_catboost_model():
    """
    CatBoost 直接使用 DataFrame + cat_features 索引
    """
    return CatBoostRegressor(
        depth=8,
        learning_rate=0.05,
        n_estimators=800,
        loss_function="RMSE",
        eval_metric="RMSE",
        random_seed=42,
        verbose=False,
    )


# =========================================================
# 5. 训练 + 验证（2020-2023 训练，2024 验证）
# =========================================================
def train_and_validate_models(df_lagged: pd.DataFrame):
    """
    用带 lag 特征的数据，训练 XGB / LGBM / RF / CatBoost，并在 2024 做验证。
    返回：
      - metrics_all：字典，包含各模型的指标
      - 已训练好的 CatBoost（后续可选直接用）
    """
    # 删除 lag 特征或标签为 NaN 的行
    df_clean = df_lagged.dropna(subset=FEATURE_COLS + ["sales_volume"]).copy()

    # 按年份切分
    train_df = df_clean[df_clean["year"] < 2024]
    valid_df = df_clean[df_clean["year"] == 2024]

    X_train = train_df[FEATURE_COLS]
    y_train = train_df["sales_volume"].values

    X_valid = valid_df[FEATURE_COLS]
    y_valid = valid_df["sales_volume"].values

    print(
        f"Shape train (with lag): {X_train.shape}, "
        f"Shape valid (with lag): {X_valid.shape}\n"
    )

    metrics_all = {}

    # ---------- XGBoost ----------
    xgb_model = build_xgb_pipeline()
    xgb_model.fit(X_train, y_train)
    y_pred_xgb = xgb_model.predict(X_valid)
    metrics_xgb = compute_metrics(y_valid, y_pred_xgb)
    metrics_all["xgb"] = metrics_xgb

    print("Model: XGBoost")
    print(
        f"  MAE      : {metrics_xgb['MAE']:.2f} ({metrics_xgb['MAE_pct']:.2f}%)\n"
        f"  RMSE     : {metrics_xgb['RMSE']:.2f} ({metrics_xgb['RMSE_pct']:.2f}%)\n"
        f"  MedianAE : {metrics_xgb['MedianAE']:.2f} ({metrics_xgb['MedianAE_pct']:.2f}%)\n"
        f"  MAPE     : {metrics_xgb['MAPE']:.2f}%\n"
        f"  R2       : {metrics_xgb['R2']:.4f}\n"
    )

    # ---------- LightGBM ----------
    lgbm_model = build_lgbm_pipeline()
    lgbm_model.fit(X_train, y_train)
    y_pred_lgbm = lgbm_model.predict(X_valid)
    metrics_lgbm = compute_metrics(y_valid, y_pred_lgbm)
    metrics_all["lgbm"] = metrics_lgbm

    print("Model: LightGBM")
    print(
        f"  MAE      : {metrics_lgbm['MAE']:.2f} ({metrics_lgbm['MAE_pct']:.2f}%)\n"
        f"  RMSE     : {metrics_lgbm['RMSE']:.2f} ({metrics_lgbm['RMSE_pct']:.2f}%)\n"
        f"  MedianAE : {metrics_lgbm['MedianAE']:.2f} ({metrics_lgbm['MedianAE_pct']:.2f}%)\n"
        f"  MAPE     : {metrics_lgbm['MAPE']:.2f}%\n"
        f"  R2       : {metrics_lgbm['R2']:.4f}\n"
    )

    # ---------- RandomForest ----------
    rf_model = build_rf_pipeline()
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_valid)
    metrics_rf = compute_metrics(y_valid, y_pred_rf)
    metrics_all["rf"] = metrics_rf

    print("Model: RandomForest")
    print(
        f"  MAE      : {metrics_rf['MAE']:.2f} ({metrics_rf['MAE_pct']:.2f}%)\n"
        f"  RMSE     : {metrics_rf['RMSE']:.2f} ({metrics_rf['RMSE_pct']:.2f}%)\n"
        f"  MedianAE : {metrics_rf['MedianAE']:.2f} ({metrics_rf['MedianAE_pct']:.2f}%)\n"
        f"  MAPE     : {metrics_rf['MAPE']:.2f}%\n"
        f"  R2       : {metrics_rf['R2']:.4f}\n"
    )

    # ---------- CatBoost ----------
    cat_model = build_catboost_model()
    cat_feature_indices = [X_train.columns.get_loc(col) for col in CAT_FEATURES]

    cat_model.fit(X_train, y_train, cat_features=cat_feature_indices)
    y_pred_cat = cat_model.predict(X_valid)
    metrics_cat = compute_metrics(y_valid, y_pred_cat)
    metrics_all["catboost"] = metrics_cat

    print("Model: CatBoost")
    print(
        f"  MAE      : {metrics_cat['MAE']:.2f} ({metrics_cat['MAE_pct']:.2f}%)\n"
        f"  RMSE     : {metrics_cat['RMSE']:.2f} ({metrics_cat['RMSE_pct']:.2f}%)\n"
        f"  MedianAE : {metrics_cat['MedianAE']:.2f} ({metrics_cat['MedianAE_pct']:.2f}%)\n"
        f"  MAPE     : {metrics_cat['MAPE']:.2f}%\n"
        f"  R2       : {metrics_cat['R2']:.4f}\n"
    )

    return metrics_all, cat_model


# =========================================================
# 6. 2025 特征构造 + CatBoost 预测
# =========================================================
def build_2025_features_from_lagged(df_lagged: pd.DataFrame) -> pd.DataFrame:
    """
    利用到 2024 年的 lag 特征，构造 2025 年的特征行：
    - sales_last1_2025 = 2024 的 sales_volume
    - sales_last2_2025 = 2024 的 sales_last1
    - sales_avg3_2025  = (2024, 2023, 2022) 三年平均
    价格、里程的 last1 / trend 沿用 2024。
    """
    group_cols = ["model", "region", "fuel_type", "transmission", "color"]

    df_sorted = df_lagged.sort_values(group_cols + ["year"])
    last_rows = df_sorted.groupby(group_cols, as_index=False).tail(1)

    # 过滤必须有完整 lag 信息的组合
    last_rows = last_rows.dropna(
        subset=["sales_last1", "sales_last2", "sales_avg3", "price_last1", "mileage_last1"]
    )

    rows_2025 = []

    for _, row in last_rows.iterrows():
        new_row = row.copy()
        new_row["year"] = 2025

        sales_last1_2025 = row["sales_volume"]
        sales_last2_2025 = row["sales_last1"]
        sales_avg3_2025 = (
            row["sales_volume"] + row["sales_last1"] + row["sales_last2"]
        ) / 3

        new_row["sales_last1"] = sales_last1_2025
        new_row["sales_last2"] = sales_last2_2025
        new_row["sales_avg3"] = sales_avg3_2025

        if not pd.isna(sales_last2_2025) and sales_last2_2025 != 0:
            new_row["sales_growth"] = (
                sales_last1_2025 - sales_last2_2025
            ) / sales_last2_2025
        else:
            new_row["sales_growth"] = 0.0

        new_row["price_last1"] = row["price_usd"]
        new_row["mileage_last1"] = row["mileage_km"]

        # 沿用原来的趋势（你也可以重算）
        new_row["price_trend"] = row.get("price_trend", 0.0)
        new_row["mileage_trend"] = row.get("mileage_trend", 0.0)

        # 2025 年真实销量未知
        new_row["sales_volume"] = np.nan

        rows_2025.append(new_row)

    feat_2025 = pd.DataFrame(rows_2025)
    # 确保列顺序与 FEATURE_COLS 一致
    feat_2025 = feat_2025[[c for c in feat_2025.columns if c not in ["sales_volume"]] + ["sales_volume"]]

    return feat_2025


def retrain_catboost_and_predict_2025(df_lagged: pd.DataFrame):
    """
    使用 2020-2024 全部数据重训 CatBoost，
    对构造出的 2025 特征进行预测，并按多个维度聚合。
    """
    df_clean = df_lagged.dropna(subset=FEATURE_COLS + ["sales_volume"]).copy()

    X_full = df_clean[FEATURE_COLS]
    y_full = df_clean["sales_volume"].values

    feat_2025 = build_2025_features_from_lagged(df_lagged)
    X_2025 = feat_2025[FEATURE_COLS]

    cat_model = build_catboost_model()
    cat_feature_indices = [X_full.columns.get_loc(col) for col in CAT_FEATURES]
    cat_model.fit(X_full, y_full, cat_features=cat_feature_indices)

    preds_2025 = cat_model.predict(X_2025)
    total_2025 = preds_2025.sum()

    feat_2025_out = feat_2025.copy()
    feat_2025_out["pred_sales_volume_2025_catboost"] = preds_2025

    print(f"\n=== CatBoost predicted total sales volume in 2025: {total_2025:.2f} ===\n")

    # 按不同维度聚合
    col_pred = "pred_sales_volume_2025_catboost"

    by_model = (
        feat_2025_out.groupby("model")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )
    by_region = (
        feat_2025_out.groupby("region")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )
    by_fuel = (
        feat_2025_out.groupby("fuel_type")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )
    by_trans = (
        feat_2025_out.groupby("transmission")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )
    by_color = (
        feat_2025_out.groupby("color")[col_pred]
        .sum()
        .reset_index()
        .sort_values(col_pred, ascending=False)
    )

    return total_2025, feat_2025_out, by_model, by_region, by_fuel, by_trans, by_color


# =========================================================
# 7. 指标对比表 + 单张柱状图
# =========================================================
def build_metrics_table(metrics_all: dict, arima_metrics: dict) -> pd.DataFrame:
    """
    构造成 DataFrame，对比 ARIMA / XGB / LGBM / RF / CatBoost 的指标。
    """
    df = pd.DataFrame.from_dict(metrics_all, orient="index")
    df.index.name = "model"

    # 加入 ARIMA（年度总销量）
    df.loc["arima_yearly"] = arima_metrics

    # 调整行顺序（可选）
    df = df.loc[["arima_yearly", "xgb", "lgbm", "rf", "catboost"]]

    return df


def plot_metrics_bar(metrics_df: pd.DataFrame, save_path: str = None):
    """
    画一张柱状图，对比不同模型在 4 个指标上的表现：
    - MAE_pct
    - RMSE_pct
    - MedianAE_pct
    - MAPE（本来就是百分比）
    """
    models = metrics_df.index.tolist()
    x = np.arange(len(models))

    metrics_to_plot = ["MAE_pct", "RMSE_pct", "MedianAE_pct", "MAPE"]
    colors = ["#1C69D4", "#4C8BF5", "#82B1FF", "#FFB300"]
    width = 0.18

    fig, ax = plt.subplots(figsize=(10, 5))

    for i, (metric, c) in enumerate(zip(metrics_to_plot, colors)):
        vals = metrics_df[metric].values
        ax.bar(x + i * width - 1.5 * width, vals, width, label=metric, color=c)

    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=15)
    ax.set_ylabel("Error / Percentage")
    ax.set_title("Model Performance Comparison on 2024 (with lag features)")

    ax.legend()
    ax.grid(axis="y", linestyle="--", alpha=0.3)

    fig.tight_layout()
    if save_path:
        fig.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close(fig)


# =========================================================
# 8. 主流程
# =========================================================
if __name__ == "__main__":
    # 1. 读取 & 聚合
    agg_df = load_and_aggregate(FILE_PATH)
    print("Aggregated shape (before lag):", agg_df.shape)
    print(agg_df.head(), "\n")

    # 2. ARIMA 年度基线（总销量）
    arima_metrics_2024, arima_2025_pred = arima_baseline_yearly(agg_df)

    # 3. 加 lag 特征
    agg_df_lagged = add_lag_features(agg_df)
    print(
        "Shape after adding lag features (before dropna):",
        agg_df_lagged.shape,
        "\n",
    )

    # 4. 训练 + 验证（2020-2023 训练，2024 验证）
    metrics_all, cat_model_trained = train_and_validate_models(agg_df_lagged)

    # 5. 对比表格（含百分率）
    metrics_df = build_metrics_table(metrics_all, arima_metrics_2024)
    print("\n=== Model comparison on 2024 validation (with lag, row-level vs yearly) ===")
    print(metrics_df, "\n")

    # 6. 一张柱状图比较不同指标
    plot_metrics_bar(metrics_df, save_path="model_performance_2024_with_lag.png")
    print("Saved bar chart to: model_performance_2024_with_lag.png\n")

    # 7. 用 CatBoost 预测 2025 总销量 & 分组销量
    (
        total_2025_cat,
        df_cat_2025,
        by_model_2025,
        by_region_2025,
        by_fuel_2025,
        by_trans_2025,
        by_color_2025,
    ) = retrain_catboost_and_predict_2025(agg_df_lagged)

    print("CatBoost - 2025 predicted sales by model:")
    print(by_model_2025.head(), "\n")

    print("CatBoost - 2025 predicted sales by region:")
    print(by_region_2025, "\n")

    print("CatBoost - 2025 predicted sales by fuel_type:")
    print(by_fuel_2025, "\n")

    print("CatBoost - 2025 predicted sales by transmission:")
    print(by_trans_2025, "\n")

    print("CatBoost - 2025 predicted sales by color:")
    print(by_color_2025, "\n")

    print(f"ARIMA (yearly total) predicted 2025 sales: {arima_2025_pred:.2f}")
    print(f"CatBoost (row-level) predicted 2025 sales: {total_2025_cat:.2f}")


Aggregated shape (before lag): (10306, 10)
   year     model  region fuel_type transmission   color  engine_size_l  \
0  2020  3 Series  Africa    Diesel    Automatic   Black           2.90   
1  2020  3 Series  Africa    Diesel    Automatic    Blue           3.00   
2  2020  3 Series  Africa    Diesel    Automatic     Red           3.55   
3  2020  3 Series  Africa    Diesel    Automatic  Silver           3.30   
4  2020  3 Series  Africa    Diesel       Manual   Black           4.60   

   mileage_km  price_usd  sales_volume  
0     54293.0   100920.0          8279  
1     22846.0    79435.5          6282  
2     85078.5   106306.0         14847  
3    182077.0    59951.0          2355  
4    107799.0    75078.0          3975   

=== ARIMA baseline on yearly total sales ===
year
2020    16310843
2021    16884666
2022    17920946
2023    16268654
2024    17527854
Name: sales_volume, dtype: int64 

ARIMA predicted 2024 total sales: 16227225.32
Actual 2024 total sales        : 17527854.